In [9]:
# Dependencies
import json
from pprint import pprint
from datetime import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from config import (consumer_key, consumer_secret,
                    access_token, access_token_secret)

In [10]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit=False, wait_on_rate_limit_notify=False)

# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [11]:
movie_list = ["#Gringo", "#TheHurricantHeist", "#StrangersPreyAtNight", "#AWrinkleinTime", "#LoveSimon", "#TombRaider", 
              "#PacificRimUprising", "#SherlockGnomes", "#TylerPerrysAcrimony", "#ReadyPlayerOne"]

In [12]:
# Create list of dictionaries
sentiment = []

# Date
search_date = input("What date do you want to query the movies? Has to be in this format (%Y-%m-%d): ")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# Analyze each movie in list
for movie in movie_list:
    
    # Assign title as search term
    search_term = movie
    
    # Create variable for holding the oldest tweet
    oldest_tweet = None

    # List to hold average compound values for each movie
    compound_list = []
    
    try:
        
        # Loop through 18 times (total of 1800 tweets)
        for x in range(18):
    
            # Get all tweets from home feed (for each page specified)
            public_tweets = api.search(search_term, 
                                        count=100,
                                        lang='en', 
                                        until=search_date, 
                                        max_id=oldest_tweet)

            # Loop through all tweets
            for tweet in public_tweets['statuses']:
        
                # Use filters to check if user meets conditions
                if (tweet["user"]["followers_count"] < max_followers and
                    tweet["user"]["statuses_count"] > min_tweets and
                    tweet["user"]["statuses_count"] < max_tweets and
                    tweet["user"]["friends_count"] < max_following and
                    tweet["user"]["lang"] == lang):
        
                    # Grab tweet data
                    tweet_text = tweet['text']
            
                    # Run Vader Analysis on each tweet
                    results = analyzer.polarity_scores(tweet["text"])
                    compound = results["compound"]
                
                    # Append compound value to list
                    compound_list.append(compound)
            
                    # Reassign the the oldest tweet (i.e. the max_id)
                    oldest_tweet = int(tweet["id_str"])
        
                    # Subtract 1 so the previous oldest isn't included
                    # in the new search
                    oldest_tweet -= 1
                   
        # Store average 
        tweet_dict = {"Movie Title": search_term,
                      "Search Date": search_date,
                      "Compound": np.mean(compound_list), 
                      "Tweet Count": len(compound_list)}
    
        # Append tweet data to sentiment list
        sentiment.append(tweet_dict)
    
    except:
        raise
# except:
#     print("You exceeded the rate limit!")
#     raise

What date do you want to query the movies? Has to be in this format (%Y-%m-%d): 2018-03-07


C:\Users\Justin\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Justin\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
# Create Dataframe
movie_sent_df = pd.DataFrame(sentiment)

# Save to csv
movie_sent_df.to_csv(f'prediction_data/{search_date}_movie_sentiment.csv', encoding='utf-8', index=False)
movie_sent_df

,Compound,Movie Title,Search Date,Tweet Count
0,0.164162,#Gringo,2018-03-07,39
1,NaN,#TheHurricantHeist,2018-03-07,0
2,0.427650,#StrangersPreyAtNight,2018-03-07,2
3,0.243241,#AWrinkleinTime,2018-03-07,613
4,0.294837,#LoveSimon,2018-03-07,513
5,0.202709,#TombRaider,2018-03-07,536
6,0.179792,#PacificRimUprising,2018-03-07,600
7,0.343054,#SherlockGnomes,2018-03-07,37
8,NaN,#TylerPerrysAcrimony,2018-03-07,0
9,0.235106,#ReadyPlayerOne,2018-03-07,702
